In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "../../scripts/methods/")
from my_paste2 import paste2_align, paste2_align_ref
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time
from tqdm import tqdm
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-08-06 17:39:18.110626: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-06 17:39:19.167313: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-08-06 17:39:19.167409: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-08-06 17:39:19.167417: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [1]:
# Load data
data_folder = "../../data/BARseq/BARseq_Perturbed/"
results_folder = "./results/PASTE2/"

In [ ]:
subsample_num = 5000
  
for i in tqdm(range(1, 40)):
    slice1 = ad.read_h5ad(filename=os.path.join(data_folder,"slice_{:0>2d}.h5ad".format(i)))
    slice2 = ad.read_h5ad(filename=os.path.join(data_folder,"slice_{:0>2d}.h5ad".format(i+1)))
    
    # subsample
    subsample1 = np.random.choice(slice1.shape[0], subsample_num, replace=False) if slice1.shape[0] > subsample_num else np.arange(slice1.shape[0])
    subsample2 = np.random.choice(slice2.shape[0], subsample_num, replace=False) if slice2.shape[0] > subsample_num else np.arange(slice2.shape[0])
    slice1 = slice1[subsample1,:]
    slice2 = slice2[subsample2,:]

    spatial_key = 'perturbed_spatial'
    key_added = 'align_spatial'
    align_slices, pis = paste2_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        verbose=True,
        fixed_s=0.99,
    )
    
    # recover true R and t
    R1, t1=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm['perturbed_spatial'])
    R2, t2=solve_RT_by_correspondence(align_slices[0].obsm['perturbed_spatial'], align_slices[0].obsm[key_added])
    t = t1 @ R2.T + t2
    R = R2 @ R1
    results = {"R":R, "t":t, "subsample": [subsample1, subsample2]}
    np.save(os.path.join(results_folder, "slice_{}_{}.npy".format(i,i+1)), results, allow_pickle=True)

  0%|                                                           | 0/39 [00:00<?, ?it/s]/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.147609e+00|0.000000e+00|0.000000e+00
    1|1.350376e+00|1.330914e+00|1.797234e+00
    2|1.136432e+00|1.882589e-01|2.139435e-01
    3|1.104291e+00|2.910563e-02|3.214109e-02
    4|1.103630e+00|5.992265e-04|6.613242e-04
    5|1.103615e+00|1.287314e-05|1.420700e-05
    6|1.103615e+00|5.698256e-07|6.288680e-07
 1000|1.103615e+00|0.000000e+00|0.000000e+00


  3%|█▏                                              | 1/39 [02:15<1:25:54, 135.64s/it]/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/project/Benchmark_alignment_methods/notebooks/BARSeq/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|3.000175e+00|0.000000e+00|0.000000e+00
    1|1.275963e+00|1.351303e+00|1.724213e+00
    2|1.120547e+00|1.386965e-01|1.554159e-01
    3|1.098632e+00|1.994693e-02|2.191435e-02
    4|1.098222e+00|3.740272e-04|4.107648e-04
    5|1.098205e+00|1.511833e-05|1.660303e-05
    6|1.098203e+00|1.495847e-06|1.642745e-06
    7|1.098201e+00|2.331764e-06|2.560745e-06
    8|1.098201e+00|6.690971e-08|7.348030e-08
